In [53]:
# preparation ...
import pandas as pd
from tqdm import tqdm
import itertools
from collections import Counter

# we can not find 'PanGU-Σ', 'Luminous' in close Source index, cannot find 'Galatica', 'YaLM', 'PanGu-α' in open source list , so delete them.
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala','GPT-2','GPT2','GPT 2']
closeSourceList = ['GShard','GPT-3','GPT 3', 'GPT3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4', 'GPT4','Claude']

readfile = "/data/jx4237data/DataForChatGPTinnovationWaves/"
df = pd.read_csv(readfile + 'LLM09enhanced.csv',dtype=object)
# some paper we need to delete:
# “fake” palm paper 1909.02134
# fake openai 1506.04006
delete_list = ['1506.04006','1909.02134']
df = df[~df['id'].isin(delete_list)]
# model2paper
model2paper = {}
for model in tqdm(openSourceList+closeSourceList):
    if model in model2paper:
        print('error')
    model2paper[model] = set()
    for paper in df[df['abstract'].str.contains("\\b%s\\b"%model) | df['abstract'].str.lower().str.contains("\\b%s\\b"%model.lower())  | df['title'].str.contains("\\b%s\\b"%model) | df['title'].str.lower().str.contains("\\b%s\\b"%model.lower())]['id']  :
        model2paper[model].add(paper)

for i in model2paper['GPT 4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT 3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT2']:
    model2paper['GPT-2'].add(i)
for i in model2paper['GPT 2']:
    model2paper['GPT-2'].add(i)
del model2paper['GPT 4']
del model2paper['GPT4']
del model2paper['GPT3']
del model2paper['GPT 3']
del model2paper['GPT2']
del model2paper['GPT 2']

# paper2model
paper2model = {}
for model, paperSet in model2paper.items():
    for paper in paperSet:
        if paper not in paper2model:
            paper2model[paper] =[model]
        else:
            paper2model[paper].append(model)

openPaperSet = set()
closePaperSet = set()
for k,v in model2paper.items():
    if k in closeSourceList:
        for paper in v:
            closePaperSet.add(paper)
    if k in openSourceList:
        for paper in v:
            openPaperSet.add(paper)
pureOpen = openPaperSet - closePaperSet
pureClose = closePaperSet - openPaperSet
mixed = openPaperSet & closePaperSet
all = openPaperSet | closePaperSet

paper2pubdate = {}
for i in range(len(df)):
    paper2pubdate[list(df['id'])[i]] = list(df['update_date'])[i]
import warnings
warnings.filterwarnings('ignore')

100%|██████████| 59/59 [00:20<00:00,  2.87it/s]


In [54]:
from collections import Counter
# 记录pureOpen每个月份，提及top model的数量
# 月份，top model, count
pureOpenModelTrend = {}
for paper in all:
    if paper2pubdate[paper] not in pureOpenModelTrend:
        tempCounter = Counter()
        for modelname in paper2model[paper]:
            tempCounter[modelname] += 1
        pureOpenModelTrend[paper2pubdate[paper]] = tempCounter
    else:
        for modelname in paper2model[paper]:
            pureOpenModelTrend[paper2pubdate[paper]][modelname] += 1

# full time
temp = Counter()
for k,v in pureOpenModelTrend.items():
    temp = temp + v
print('all    ',sorted(temp.items(),key=lambda x : -x[1]))

for k,v in sorted(pureOpenModelTrend.items(),key=lambda x : x[0]):
    print(k,sorted(v.items(),key=lambda x : -x[1]))


all     [('ChatGPT', 1264), ('GPT-3', 1027), ('GPT-4', 636), ('GPT-2', 552), ('T5', 307), ('LLaMA', 246), ('Codex', 127), ('OPT', 93), ('PaLM', 87), ('FLAN', 78), ('BLOOM', 74), ('Flan-T5', 57), ('Alpaca', 55), ('InstructGPT', 54), ('Vicuna', 51), ('Claude', 37), ('CodeGen', 27), ('mT5', 24), ('T0', 20), ('BLOOMZ', 19), ('Pythia', 15), ('GLM', 13), ('Anthropic', 11), ('LaMDA', 10), ('Chinchilla', 9), ('AlphaCode', 9), ('ChatGLM', 9), ('UL2', 8), ('NLLB', 8), ('Flan-PaLM', 7), ('Tk-Instruct', 6), ('GLaM', 6), ('GShard', 6), ('HyperCLOVA', 5), ('WebGPT', 5), ('Koala', 5), ('Gopher', 5), ('Cohere', 4), ('U-PaLM', 4), ('Jurassic-1', 4), ('mT0', 3), ('MT-NLG', 3), ('GPT-NeoX-20B', 3), ('ERNIE 3', 3), ('AlexaTM', 3), ('Sparrow', 2), ('Flan-U-PaLM', 2), ('CodeGeeX', 1), ('WeLM', 1), ('OPT-IML', 1), ('ERNIE 3.0 Titan', 1), ('Yuan 1.0', 1), ('CPM-2', 1)]
2019-06-12 [('GPT-2', 1)]
2019-06-14 [('GPT-2', 1)]
2019-06-18 [('GPT-2', 1)]
2019-06-20 [('GPT-2', 1)]
2019-07-04 [('GPT-2', 1)]
2019-08-06 [

In [55]:
from tqdm import tqdm
from dateutil import parser
flag = 0
for temp_model in tqdm(temp.keys()):
    temp_dict = {}
    for k,v in pureOpenModelTrend.items():
        temp_dict[parser.parse(k)] = v[temp_model]
    sf = pd.Series(temp_dict)
    sf = sf.sort_index()
    # if temp_model in openSourceList:
    #     temp_model = temp_model+' (Public Available)'
    # else:
    #     temp_model = temp_model+' (Closed Source)'
    if flag == 0:
        df_cumu = pd.DataFrame({temp_model:sf.values},index = sf.index)
        flag = 1
    else:
        df_cumu[temp_model] = sf

  0%|          | 0/53 [00:00<?, ?it/s]

100%|██████████| 53/53 [00:01<00:00, 44.36it/s]


In [56]:
df_for_rolling = pd.DataFrame()
df_for_sum = pd.DataFrame()

openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala','GPT-2']
closeSourceList = ['GShard','GPT-3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4','Claude']


df_for_sum['closed model'] = df_cumu[closeSourceList].sum(axis=1)
df_for_sum['open model'] = df_cumu[openSourceList].sum(axis=1)

for model_name in df_cumu.columns:
    df_for_rolling['%s ( 10 days rolling average )'%model_name] = df_cumu[model_name].rolling('10D').mean()
df_for_rolling['Open Sourced model ( 10 days rolling average )'] = df_for_sum['open model'].rolling('10D').mean()
df_for_rolling['Closed Sourced Model ( 10 days rolling average )'] = df_for_sum['closed model'].rolling('10D').mean()

df_cumu['date'] = df_cumu.index
df_for_rolling['date'] = df_cumu.index
df_for_sum['date'] = df_cumu.index

In [57]:
# the index is time, we can do many time-series operation
df_cumu['2021-1':]

,GPT-3,PaLM,ChatGPT,GPT-4,LaMDA,T5,GPT-2,Codex,LLaMA,Anthropic,...,Gopher,Sparrow,Flan-U-PaLM,WeLM,OPT-IML,AlexaTM,ERNIE 3.0 Titan,Yuan 1.0,CPM-2,date
2021-01-01,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-01-01
2021-01-05,1,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-01-05
2021-01-13,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-01-13
2021-01-19,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-01-19
2021-01-25,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-02,7,2,10,12,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,2023-10-02
2023-10-03,8,1,12,18,0,1,1,0,9,0,...,0,0,0,0,0,0,0,0,0,2023-10-03
2023-10-04,5,1,3,5,0,2,1,0,4,0,...,0,0,1,0,0,0,0,0,0,2023-10-04
2023-10-05,4,0,5,7,0,0,0,1,3,0,...,0,0,0,0,0,0,0,0,0,2023-10-05


In [58]:
# top 15 mentioned models
top_mentioned = list(df_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])

In [59]:
import plotly.express as px
df = df_for_rolling[['Open Sourced model ( 10 days rolling average )','Closed Sourced Model ( 10 days rolling average )']]
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open sourced v.s Closed sourced: ChatGPT is the gamechanger?',)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2022-11-30: ChatGPT launched ! ", annotation_position="top right",
              fillcolor="green", opacity=0.5, line_width=1, )

fig.show()

In [60]:
df = df_for_sum[['closed model', 'open model']].cumsum()
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open sourced v.s Closed sourced: Before ChatGPT, GPT-3 drew more attention than t-5. (Cumulative Distribution)',
              log_y=True,
              )
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2020-11-30: ChatGPT", annotation_position="top right",
              fillcolor="green", opacity=0.25, line_width=1,)



fig.add_vrect(x0="2019-11-21", x1="2019-12-02",
              annotation_text="2019-10-23: T5 model", annotation_position="top left",
              fillcolor="grey", opacity=0.25, line_width=1)


fig.add_vrect(x0="2020-06-06", x1="2020-06-20",
              annotation_text="2020-06-11: GPT-3 model", annotation_position="top left",
              fillcolor="blue", opacity=0.25, line_width=1)


fig.show()

In [77]:
top_mentioned = list(df_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_for_rolling[top_mentioned]
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open source: T5 (Google), OPT (Meta), and LLaMA (Meta) \n\n Closed Source: GPTs (GPT-3, CodeX, ChatGPT,GPT-4) plus PaLM (Google)',
)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2022-11-30: ChatGPT launched ! ", annotation_position="bottom left",
              fillcolor="green", opacity=0.5, line_width=1, )

fig.show()

In [78]:
top_mentioned = list(df_cumu.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_cumu[top_mentioned].cumsum()
df['date'] = df.index
fig = px.line(df, x='date', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Open: T5 (Google), OPT (Meta), BLOOM (HuggingFace), LLaMA (Meta), Flan (Google), Flan-T5(T5);<br>Closed: GPTs (GPT-3, CodeX, InsturctGPT, ChatGPT,GPT-4) plus PaLM (Google)',
              log_y= True
)

fig.add_vrect(x0="2022-11-25", x1="2022-12-06",
              annotation_text="2022-11-30: ChatGPT launched ! ", annotation_position="top left",
              fillcolor="green", opacity=0.5, line_width=1, )

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

#### Let's consider the diffusion speed of these models. Yes, do an alignment of the above plots.

X-axes are the days after the model was proposed.

In [63]:
# the first time when a model is proposed.
modelLaunchTime = {}

In [64]:
for paper, modellist in paper2model.items():
    for model in modellist:
        temp_time = parser.parse(paper2pubdate[paper])
        if model not in modelLaunchTime:
            modelLaunchTime[model] = temp_time
        else:
            # if temp_time is earlier, temp_time is potential the launch time.
            if modelLaunchTime[model] > temp_time:
                modelLaunchTime[model] = temp_time


In [65]:
# record each datedelta(days), how many papers mentioned regarding model name
# timedelta, modelname, count
ModelTrend = {}
for paper in all:
    for modelname in paper2model[paper]:
        timedelta = parser.parse(paper2pubdate[paper]) - modelLaunchTime[modelname]
        if timedelta not in ModelTrend:
            tempCounter = Counter()
            tempCounter[modelname] += 1
            ModelTrend[timedelta] = tempCounter
        else:
            ModelTrend[timedelta][modelname] += 1

# full time
temp = Counter()
for k,v in ModelTrend.items():
    temp = temp + v
print('all    ',sorted(temp.items(),key=lambda x : -x[1]))

for k,v in sorted(ModelTrend.items(),key=lambda x : x[0]):
    print(k,sorted(v.items(),key=lambda x : -x[1]))


all     [('ChatGPT', 1264), ('GPT-3', 1027), ('GPT-4', 636), ('GPT-2', 552), ('T5', 307), ('LLaMA', 246), ('Codex', 127), ('OPT', 93), ('PaLM', 87), ('FLAN', 78), ('BLOOM', 74), ('Flan-T5', 57), ('Alpaca', 55), ('InstructGPT', 54), ('Vicuna', 51), ('Claude', 37), ('CodeGen', 27), ('mT5', 24), ('T0', 20), ('BLOOMZ', 19), ('Pythia', 15), ('GLM', 13), ('Anthropic', 11), ('LaMDA', 10), ('AlphaCode', 9), ('ChatGLM', 9), ('Chinchilla', 9), ('UL2', 8), ('NLLB', 8), ('Flan-PaLM', 7), ('GShard', 6), ('GLaM', 6), ('Tk-Instruct', 6), ('Koala', 5), ('HyperCLOVA', 5), ('Gopher', 5), ('WebGPT', 5), ('Jurassic-1', 4), ('U-PaLM', 4), ('Cohere', 4), ('mT0', 3), ('GPT-NeoX-20B', 3), ('ERNIE 3', 3), ('AlexaTM', 3), ('MT-NLG', 3), ('Sparrow', 2), ('Flan-U-PaLM', 2), ('CodeGeeX', 1), ('ERNIE 3.0 Titan', 1), ('Yuan 1.0', 1), ('WeLM', 1), ('CPM-2', 1), ('OPT-IML', 1)]
0:00:00 [('GPT-4', 2), ('GLaM', 1), ('FLAN', 1), ('HyperCLOVA', 1), ('Chinchilla', 1), ('GLM', 1), ('mT0', 1), ('CodeGeeX', 1), ('T0', 1), ('G

In [66]:
flag = 0
for temp_model in tqdm(temp.keys()):
    temp_dict = {}
    for k,v in ModelTrend.items():
        temp_dict[k] = v[temp_model]
    sf = pd.Series(temp_dict)
    sf = sf.sort_index()
    if flag == 0:
        df_delta = pd.DataFrame({temp_model:sf.values},index = sf.index)
        flag = 1
    else:
        df_delta[temp_model] = sf

100%|██████████| 53/53 [00:00<00:00, 283.11it/s]


In [67]:
df_delta_for_rolling = pd.DataFrame()
df_delta_for_sum = pd.DataFrame()


for model_name in df_delta.columns:
    df_delta_for_rolling['%s ( 10 days rolling average )'%model_name] = df_delta[model_name].rolling('10D').mean()

df_delta['date'] = df_delta.index.days
df_delta_for_rolling['date'] = df_delta.index.days
df_delta_for_sum['date'] = df_delta.index.days

In [87]:
top_mentioned = list(df_delta_for_rolling.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_delta_for_rolling[['date','ChatGPT ( 10 days rolling average )','GPT-4 ( 10 days rolling average )','LLaMA ( 10 days rolling average )','GPT-3 ( 10 days rolling average )']]
df['days'] = df.index.days
fig = px.line(df, x='days', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Explosive Growth: GPT-4, ChatGPT, LLaMA',
)
fig.show()

In [69]:
top_mentioned = list(df_delta.sum(numeric_only=True, axis=0).sort_values(ascending=False).index[:10])
df = df_delta[top_mentioned].cumsum()
df['days'] = df.index.days
fig = px.line(df, x='days', y=df.columns,
              hover_data={'date': "|%B %d, %Y"},
              title='Explosive Growth: GPT-4, ChatGPT, LLaMA;   Founders: GPT-3 and T5',
              log_y= True
)
fig.show()